In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [237]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import RFECV

%matplotlib inline

# Read the CSV and Perform Basic Data Cleaning

In [238]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
y = df['koi_disposition']
df = df.drop(columns=['koi_disposition'])
# Drop the null rows
df.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [239]:
df.columns

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk',
       'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1',
       'koi_impact_err2', 'koi_duration', 'koi_duration_err1',
       'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2',
       'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_insol',
       'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg',
       'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 'koi_srad_err1',
       'koi_srad_err2', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')

A full description of the columns here: https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html



In [241]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_kepmag', 'koi_period', 
                        'koi_duration', 'ra', 'dec', 'koi_slogg', 'koi_prad', 'koi_insol',
                        'koi_srad', 'koi_steff', 'koi_teq', 'koi_depth',]]


# Create a Train Test Split

Use `koi_disposition` for the y values

In [242]:
from sklearn.model_selection import train_test_split

In [243]:
X = pd.get_dummies(selected_features)

In [244]:
X

,koi_kepmag,koi_period,koi_duration,ra,dec,koi_slogg,koi_prad,koi_insol,koi_srad,koi_steff,koi_teq,koi_depth
0,15.347,54.418383,4.50700,291.93423,48.141651,4.467,2.83,9.11,0.927,5455,443,874.8
1,15.436,19.899140,1.78220,297.00482,48.134129,4.544,14.60,39.30,0.868,5853,638,10829.0
2,15.597,1.736952,2.40641,285.53461,48.285210,4.564,33.46,891.96,0.791,5805,1395,8079.2
3,15.509,2.525592,1.65450,288.75488,48.226200,4.438,2.75,926.16,1.046,6031,1406,603.3
4,15.714,4.134435,3.14020,296.28613,48.224670,4.486,2.77,427.65,0.972,6046,1160,686.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6986,14.478,8.589871,4.80600,298.74921,46.973351,4.296,1.11,176.40,1.088,5638,929,87.7
6987,14.082,0.527699,3.22210,297.18875,47.093819,4.529,29.35,4500.53,0.903,5638,2088,1579.2
6988,14.757,1.739849,3.11400,286.50937,47.163219,4.444,0.72,1585.81,1.031,6119,1608,48.5
6989,15.385,0.681402,0.86500,294.16489,47.176281,4.447,1.07,5713.41,1.041,6173,2218,103.6


In [245]:
y

0            CONFIRMED
1       FALSE POSITIVE
2       FALSE POSITIVE
3            CONFIRMED
4            CONFIRMED
             ...      
6986    FALSE POSITIVE
6987    FALSE POSITIVE
6988         CANDIDATE
6989    FALSE POSITIVE
6990    FALSE POSITIVE
Name: koi_disposition, Length: 6991, dtype: object

In [246]:
#Split data into training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)

In [247]:
X_train.head()

,koi_kepmag,koi_period,koi_duration,ra,dec,koi_slogg,koi_prad,koi_insol,koi_srad,koi_steff,koi_teq,koi_depth
4002,15.801,99.673478,0.9680,293.05801,45.248821,4.777,576.14,1.11,0.492,4932,262,2496.0
4246,15.653,0.592244,1.3860,290.28094,45.464260,4.664,6.77,1361.22,0.591,4920,1551,3992.4
548,14.039,9.991625,3.8552,301.04239,45.022888,4.338,38.93,160.14,1.096,5874,907,3450.5
3953,13.944,178.412990,2.9298,288.32785,38.627621,4.346,33.24,4.00,1.148,6078,361,37510.0
2362,10.961,45.294223,6.8300,285.67938,50.241299,4.347,2.26,17.75,1.044,5676,524,477.1


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [248]:
# Scale or Normalize your data. Use MinMaxScaler i

from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)


In [249]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled  = X_scaler.transform(X_test)


In [258]:
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

model_training_score = round(model.score(X_train_scaled, y_train)*100,2)
base_accuracy        = round(model.score(X_test_scaled, y_test)  *100,2)

print(f"Training Data Score: {model_training_score} %")
print(f"Testing Data Score : {base_accuracy} %")

Training Data Score: 59.16 %
Testing Data Score : 58.81 %


/Users/annmcnamara/opt/anaconda3/envs/pydata/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [262]:
#Convert Categorical data to numbers using Integer or Binary Encoding

y_floats = pd.get_dummies(y)
y_floats

,CANDIDATE,CONFIRMED,FALSE POSITIVE
0,0,1,0
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0
...,...,...,...
6986,0,0,1
6987,0,0,1
6988,1,0,0
6989,0,0,1


In [266]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled  = X_scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

model_training_score = round(model.score(X_train_scaled, y_train)*100,2)
base_accuracy        = round(model.score(X_test_scaled, y_test)  *100,2)

print(f"Training Data Score: {model_training_score} %")
print(f"Testing Data Score : {base_accuracy} %")

Training Data Score: 59.16 %
Testing Data Score : 58.81 %


/Users/annmcnamara/opt/anaconda3/envs/pydata/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Train the Model



In [259]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.5916460041960709
Testing Data Score: 0.5881006864988558


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [272]:
# Create the SVC Model
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [273]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
# List the best parameters for this dataset

print(grid.best_params_)
print(grid.best_score_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=['Candidate', 'Confirmed', 'False Positive']))


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)